In [1]:
#%matplotlib inline
import numpy as np
import pandas as pd
import scipy.sparse
from sklearn import preprocessing
import xgboost as xgb
import pickle
#import graphviz
import matplotlib.pyplot as plt
%matplotlib inline

Read in just the testing and training data.

In [9]:
error = pd.read_pickle('scripts/cv_results/actions_e20/errors_search2.p')
#error['mean'] = error.apply(np.mean, axis = 1)
# error = error.astype(np.float64)
# error['mean'] = error.mean(1)
# error['std'] = error.std(1)

In [10]:
error

,test-error-mean0,test-error-std0,train-error-mean0,train-error-std0
0,0.174344,0.005119,0.150627,0.005498
1,0.170641,0.004061,0.147590,0.005283
2,0.169299,0.003165,0.146437,0.005372
3,0.167820,0.002586,0.145595,0.005025
4,0.166345,0.001795,0.144863,0.004747
5,0.165601,0.001146,0.144355,0.004893
6,0.165161,0.000934,0.144190,0.004910
7,0.164800,0.000748,0.143610,0.004713
8,0.164619,0.000757,0.143810,0.004729
9,0.164205,0.000594,0.143480,0.004844


In [27]:
errorerror.astype('float')

,0,0,0,0,0,0
0,0.167572,0.167466,0.167727,0.167496,0.167596,0.168133
1,0.166168,0.165801,0.166565,0.166062,0.166788,0.167285
2,0.165645,0.165012,0.165481,0.165554,0.165830,0.166340
3,0.165161,0.164656,0.165096,0.164939,0.165238,0.165888
4,0.164776,0.164280,0.164733,0.164440,0.164568,0.165553
5,0.164559,0.164026,0.164268,0.164253,0.164273,0.164967
6,0.163968,0.163701,0.163974,0.163812,0.163975,0.164313
7,0.163879,0.163163,0.163642,0.163537,0.163699,0.163967
8,0.163576,0.163040,0.163336,0.163214,0.163468,0.163660
9,0.163320,0.162803,0.163280,0.162983,0.163203,0.163508


In [15]:
error3.to_pickle('scripts/cv_results/actions_e20/params1_err.p')

In [3]:
#train_100 = pd.read_csv('Airbnb_data/train_users_2.csv', nrows = 100, index_col='id')
train_data = pd.read_csv('data/train_users_2.csv', index_col = 'id')
test_data = pd.read_csv('data/test_users.csv', index_col = 'id')
#sessions = pd.read_csv('../data/sessions.csv')
#age = pd.read_csv('Airbnb_data/age_gender_bkts.csv')
#countries = pd.read_csv('Airbnb_data/countries.csv')
all_df = pd.read_pickle('data/all_df.p')

In [163]:
for i in np.unique(test_data.first_browser):
    if i not in np.unique(train_data.first_browser):
        print i

IBrowse
Nintendo Browser
UC Browser


In [142]:
#pd.isnull(sessions.groupby('user_id')['secs_elapsed'].sum()) == False
sessions.loc[sessions.user_id == '00allnceb8',:]
#sessions.loc[sessions.user_id == 'd1mm9tcy42',:]

,user_id,action,action_type,action_detail,device_type,secs_elapsed
1150481,00allnceb8,create,submit,signup,-unknown-,NaN


In [8]:
#find most popular actions
sessions.groupby('user_id').count()

NameError: name 'sessions' is not defined

In [5]:
error = pd.read_pickle('scripts/cv_results.p')
params = pd.read_pickle('scripts/cv_params.p')

In [6]:
res = pd.concat([error, params], axis = 1)

In [7]:
res.sort_values(by = 'test-error-mean', axis = 0);

In [3]:
all_df = feakaggle_model(all_df)

#one hot encoding of features
print 'number of columns before one hot encoding', all_df.shape[1]
ohe_feats = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']
all_df = pd.get_dummies(all_df, columns = ohe_feats)
print 'number of columns after one hot encoding', all_df.shape[1]

number of columns before one hot encoding 17
number of columns after one hot encoding 161


In [6]:
with open('xgbmodels/sessions_e20_25n.p', 'rb') as f:
    bst = pickle.load(f)
features = sorted(bst.get_fscore().items(), key = lambda x: x[1], reverse = True)
[(feat, all_df.columns[int(feat[1:])], score) for feat, score in features]

NameError: name 'all_df' is not defined

In [ ]:
xgb.to_graphviz(bst, num_trees=11)

In [ ]:
submission = pd.DataFrame(np.column_stack([idx, cntr]), columns = ['id', 'country'])
submission.to_csv('submissions/datebin_model.csv', index=False)